In [1]:
!git clone https://github.com/fayrose/EgyptianTranslation.git

Cloning into 'EgyptianTranslation'...
remote: Enumerating objects: 143, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 143 (delta 34), reused 143 (delta 34), pack-reused 0
Receiving objects: 100% (143/143), 5.88 MiB | 18.88 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [2]:
!pip install OpenNMT-py

     |████████████████████████████████| 215kB 7.5MB/s 
     |████████████████████████████████| 748.8MB 24kB/s 
     |████████████████████████████████| 102kB 12.8MB/s 
     |████████████████████████████████| 14.3MB 258kB/s 
     |████████████████████████████████| 81kB 10.9MB/s 
     |████████████████████████████████| 61kB 8.8MB/s 
     |████████████████████████████████| 81kB 11.5MB/s 
     |████████████████████████████████| 645kB 45.9MB/s 
     |████████████████████████████████| 1.2MB 41.5MB/s 
ERROR: torchvision 0.10.0+cu102 has requirement torch==1.9.0, but you'll have torch 1.6.0 which is incompatible.
  Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Found existing installation: Flask 1.1.4
    Uninstalling Flask-1.1.4:
      Successfully uninstalled Flask-1.1.4
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
  Found existing insta

In [36]:
string = """
# egy_eng.yaml

## Where the samples will be written
save_data: EgyptianTranslation/onmt-files/run/example
## Where the vocab(s) will be written
src_vocab: EgyptianTranslation/onmt-files/run/example.vocab.egy
tgt_vocab: EgyptianTranslation/onmt-files/run/example.vocab.eng
# Prevent overwriting existing files in the folder
overwrite: True

# Corpus opts:
data:
    corpus_1:
        path_src: EgyptianTranslation/compiled_corpora/aligned_train.egy.csv
        path_tgt: EgyptianTranslation/compiled_corpora/aligned_train.eng.csv
    valid:
        path_src: EgyptianTranslation/compiled_corpora/aligned_val.egy.csv
        path_tgt: EgyptianTranslation/compiled_corpora/aligned_val.eng.csv
"""
with open("egy_eng.yaml", 'w') as yaml_file:
  yaml_file.write(string)

In [37]:
!onmt_build_vocab -config egy_eng.yaml -n_sample 10000

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2021-06-24 06:35:12,268 INFO] Counter vocab from 10000 samples.
[2021-06-24 06:35:12,268 INFO] Build vocab on 10000 transformed examples/corpus.
[2021-06-24 06:35:12,276 INFO] corpus_1's transforms: TransformPipe()
[2021-06-24 06:35:12,276 INFO] Loading ParallelCorpus(EgyptianTranslation/compiled_corpora/aligned_train.egy.csv, EgyptianTranslation/compiled_corpora/aligned_train.eng.csv, align=None)...
[2021-06-24 06:35:12,377 INFO] Counters src:2579
[2021-06-24 06:35:12,377 INFO] Counters tgt:4201
[2021-06-24 06:35:12,377 WARNING] path EgyptianTranslation/onmt-files/run/example.vocab.egy exists, may overwrite...
[2021-06-24 06:35:12,380 WARNING] path EgyptianTranslation/onmt-files/run/example.vocab.eng exists, may overwrite...


In [38]:
string = """
# Vocabulary files that were just created
src_vocab: EgyptianTranslation/onmt-files/run/example.vocab.egy
tgt_vocab: EgyptianTranslation/onmt-files/run/example.vocab.eng

# Train on a single GPU
world_size: 1

# Where to save the checkpoints
save_model: EgyptianTranslation/onmt-files/run/model
save_checkpoint_steps: 1000
train_steps: 3000
valid_steps: 1500

# Optimization
# model_dtype: "fp32"
optim: "adam"
learning_rate: 2
warmup_steps: 8000
decay_method: "noam"
adam_beta2: 0.998
max_grad_norm: 0
param_init: 0
param_init_glorot: true
label_smoothing: 0.1
normalization: "tokens"

# Model
encoder_type: transformer
decoder_type: transformer
# enc_layers: 6
# dec_layers: 6
position_encoding: true
heads: 8
rnn_size: 512
word_vec_size: 512
max_generator_batches: 2
transformer_ff: 2048
dropout_steps: [0]
dropout: [0.1]
accum_count: 3
attention_dropout: [0.1]
# share_decoder_embeddings: true
# share_embeddings: true
"""

with open("egy_eng.yaml", 'a') as yaml_file:
  yaml_file.write(string)

In [ ]:
!onmt_train --config egy_eng.yaml -share_vocab -gpu_ranks 0

[2021-06-24 06:35:20,719 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2021-06-24 06:35:20,719 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2021-06-24 06:35:20,719 INFO] Missing transforms field for valid data, set to default: [].
[2021-06-24 06:35:20,719 INFO] Parsed 2 corpora from -data.
[2021-06-24 06:35:20,719 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2021-06-24 06:35:20,719 INFO] Loading vocab from text file...
[2021-06-24 06:35:20,719 INFO] Loading src vocabulary from EgyptianTranslation/onmt-files/run/example.vocab.egy
[2021-06-24 06:35:20,727 INFO] Loaded src vocab has 2579 tokens.
[2021-06-24 06:35:20,728 INFO] Loading tgt vocabulary from EgyptianTranslation/onmt-files/run/example.vocab.eng
[2021-06-24 06:35:20,739 INFO] Loaded tgt vocab has 4201 tokens.
[2021-06-24 06:35:20,741 INFO] Building fields with vocab in counters...
[2021-06-24 06:35:20,746 INFO]  * tgt vocab size: 4205.
[2021-

In [ ]:
!onmt_translate -model EgyptianTranslation/onmt-files/run/model_step_3000.pt -src /content/EgyptianTranslation/compiled_corpora/aligned_test.egy.csv -output EgyptianTranslation/onmt-files/run/pred_1000.txt -verbose
